# CISO Scenario and Agent Setup

This notebook will walk through the necessary steps to set up and run the **ITBench CISO scenario** and **CISO Agent**, developed by IBMers, within a watsonx project in Watson Studio. This notebook is specific to scenario **3.gen-cis-b-rhel9-ansible-opa** but can be used as a template to modify and run any of the other CISO scenarios. It uses forks of the two repos, one for __[Scenarios](https://github.com/ibm-client-engineering/ITBench/tree/main)__ and the other for the __[CISO Agent](https://github.com/ibm-client-engineering/ITBench-CISO-CAA-Agent)__.

This demo assumes use of the __[RHEL9 VM TechZone Environment](https://techzone.ibm.com/collection/tech-zone-certified-base-images/journey-base-vms)__, and the __[watsonx.ai/gov SaaS TechZone Environment](https://techzone.ibm.com/collection/tech-zone-certified-base-images/journey-watsonx)__, as they both run on IBM Public Cloud and the latter can access the former.

## Download and Install Prerequisites

Before beginning, you will need to upload a completed version of [the input.json file](https://github.com/ibm-client-engineering/ITBench/blob/main/ciso/3.gen-cis-b-rhel9-ansible-opa/input.json) and the ssh key for your RHEL9 VM to this project.

### Setup Required Files

In order to retreive the files we uploaded to the project, we'll need to use the ibm_boto3 client. To make this easier, go to the **</>** code snippets icon on the top right, and select "Data Ingestion" and paste it into the code block below so all the required values are prepopulated. Note: we only need the code through the definition of the 'bucket' variable.

In [1]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = ''

Download the uploaded files into the notebook's file system

In [2]:
!mkdir /home/wsuser/.ssh

In [3]:
cos_client.download_file(Bucket=bucket,Key='pem_ibmcloudvsi_download.pem',Filename='/home/wsuser/.ssh/pem_ibmcloudvsi_download.pem')
cos_client.download_file(Bucket=bucket,Key='input.json',Filename='/home/wsuser/work/input.json')

Set the correct permissions on the ssh key

In [4]:
!chmod 600 /home/wsuser/.ssh/pem_ibmcloudvsi_download.pem

#### Create necessary directories and specify locations

In [5]:
agent_workdir = "/home/wsuser/work/agent_workdir"
input_json_path = "/home/wsuser/work/input.json"
ssh_key_path = "/home/wsuser/work/pem_ibmcloudvsi_download.pem"

# Ensure the agent workdir exists
os.makedirs(agent_workdir, exist_ok=True)

# Set environment variables expected by the Makefile
os.environ['SHARED_WORKSPACE'] = agent_workdir
os.environ['INPUT_FILE'] = input_json_path
os.environ['ANSIBLE_PRIVATE_KEY_FILE'] = ssh_key_path

#### Clone the Repos
Now we will clone both the aforementioned github repositories into the filesystem of notebook so we can run the scenario setup makefiles and the agent.

In [6]:
import subprocess

In [7]:
# Define where you want the repo cloned
repo_url = "https://github.com/ibm-client-engineering/ITBench.git"
clone_dir = "/home/wsuser/work/ITBench"

# Clone only if it doesn't already exist
if not os.path.exists(clone_dir):
    subprocess.run(["git", "clone", repo_url, clone_dir])
else:
    print(f"Repository already exists at: {clone_dir}")

Cloning into '/home/wsuser/work/ITBench'...


In [8]:
# Define where you want the repo cloned
repo_url = "https://github.com/ibm-client-engineering/ITBench-CISO-CAA-Agent.git"
branch = "girish-updates"
clone_dir = "/home/wsuser/work/ITBench-CISO-CAA-Agent"

# Clone only if it doesn't already exist
if not os.path.exists(clone_dir):
    subprocess.run(["git", "clone", "-b", branch, "--single-branch", repo_url, clone_dir])
else:
    print(f"Repository already exists at: {clone_dir}")

Cloning into '/home/wsuser/work/ITBench-CISO-CAA-Agent'...


#### Edit Ansible inventory details to match RHEL host information

We need to specify the appropriate port for our TechZone environment, which is 30276 instead of the typical 22. We also need to tell ansible where the python interpreter is on the RHEL9 machine. Double-check this information with your environment details to ensure it matches.

In [9]:
%cd /home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa/

/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [10]:
%%writefile dynamic_inventory.py
#!/usr/bin/env python3
# Copyright contributors to the ITBench project. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import json
import os

input_file = os.getenv("BUNDLE_INPUT_FILE", "input.json")
with open(input_file, "r") as f:
    input = json.load(f)

inventory_host = input["inventory_host"]
target_server = input["target_server"]
host_alias = target_server["alias"]
username = target_server["username"]
server_address = target_server["address"]
server_sshkey = target_server["sshkey"]

inventory = {
    inventory_host: {
        "hosts": [host_alias],
        "vars": {
            "ansible_user": username,
            "ansible_ssh_private_key_file": server_sshkey,
            "ansible_ssh_common_args": "-o StrictHostKeyChecking=no -p 30276", # set the port for your RHEL9 VM here
        },
    },
    "_meta": {"hostvars": {host_alias: {"ansible_host": server_address}}},
}

print(json.dumps(inventory))


Overwriting dynamic_inventory.py


In [11]:
%cd /home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa/tasks

/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa/tasks


In [12]:
%%writefile user_create.yml
---
- become: true 
  block:
  - name: Add ansible user
    user:
      name: "{{ check_target_user }}"
      state: present
      shell: /bin/bash
      create_home: yes

  - name: Set password for ansible user
    user:
      name: "{{ check_target_user }}"
      password: "{{ 'created_user_password' | password_hash('sha512') }}"

  - name: Create .ssh directory
    file:
      path: "/home/{{ check_target_user }}/.ssh"
      state: directory
      mode: '0700'
      owner: "{{ check_target_user }}"
      group: "{{ check_target_user }}"

  - name: Generate SSH key pair
    openssh_keypair:
      path: "/home/{{ check_target_user }}/.ssh/id_rsa"
      owner: "{{ check_target_user }}"
      group: "{{ check_target_user }}"
      mode: '0600'
      type: rsa
      size: 2048
    register: keypair

  - name: Set up public key for SSH access
    copy:
      content: "{{ keypair.public_key }}"
      dest: "/home/{{ check_target_user }}/.ssh/authorized_keys"
      mode: '0600'
      owner: "{{ check_target_user }}"
      group: "{{ check_target_user }}"

  - name: Grant sudo privileges to ansible user for ansible-playbook command
    copy:
      dest: "/etc/sudoers.d/{{ check_target_user }}"
      content: |
        {{ check_target_user }} ALL=(ALL) NOPASSWD: /usr/bin/ansible-playbook
        {{ check_target_user }} ALL=(ALL) NOPASSWD: /bin/sh
      mode: '0440'

  - name: Output user details and private key
    debug:
      msg: |
        User created: "{{ check_target_user }}"
        Private Key:
        {{ keypair.filename }}

  - name: Fetch a file from remote to local
    fetch:
      src: "{{ keypair.filename }}"
      dest: "{{ check_target_ssh_key }}"
      flat: yes
    become: yes

  - name: Define inventory content
    set_fact:
      inventory_content: |
        [rhel9_servers]
        rhel9 ansible_host="{{ check_target_hostname }}" ansible_user="{{ check_target_user }}" ansible_ssh_private_key_file="{{ check_target_ssh_key }}" ansible_python_interpreter="{{ check_target_python_interpreter }}" ansible_ssh_common_args="-p 30276" ansible_python_interpreter="/usr/bin/python3"

  - name: Write inventory content to a file
    ansible.builtin.copy:
      content: "{{ inventory_content }}"
      dest: "{{ path_to_evaluator_inventory }}"
    delegate_to: localhost
    become: false


Overwriting user_create.yml


#### Add Agent Source Files to Path

This way, we can call the main agent function later as a python module and all of its dependences (ciso_agent) will be found and resolved

In [13]:
import sys
sys.path.append("/home/wsuser/work/ITBench-CISO-CAA-Agent/src")

#### Create a .env file for the agent

Fill in the relevant details so the agent can invoke an LLM through watsonx:

In [14]:
%cd /home/wsuser/work/ITBench-CISO-CAA-Agent/

/home/wsuser/work/ITBench-CISO-CAA-Agent


In [15]:
%%writefile .env
# .env file
LLM_BASE_URL = "https://us-south.ml.cloud.ibm.com"  # before `/ml/v1/text/generation`
LLM_API_KEY = ""
LLM_MODEL_NAME = "meta-llama/llama-3-3-70b-instruct" # "ibm/granite-3-2-8b-instruct"
WATSONX_PROJECT_ID = ""

Writing .env


#### Install and Test Open Policy Agent

The CISO Agent will use OPA Rego to run the policy files it generates for the scenario

In [16]:
%cd /home/wsuser/

/home/wsuser


In [17]:
!curl -L -o opa https://openpolicyagent.org/downloads/latest/opa_linux_amd64

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0    290      0 --:--:-- --:--:-- --:--:--   289
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 66.4M  100 66.4M    0     0  46.0M      0  0:00:01  0:00:01 --:--:--  202M


In [18]:
!chmod 755 /home/wsuser/opa

In [19]:
# Add the OPA binary location to PATH
os.environ["PATH"] = os.path.expanduser("~") + ":" + os.environ["PATH"]

# Check if the path is updated
print(os.environ["PATH"])

/home/wsuser:/opt/conda/envs/Python-RT24.1/bin:/opt/conda/condabin:/opt/conda/bin:/usr/bin:/opt/ibm/dsdriver/bin


Test that OPA is installed and working

In [20]:
!opa version

Version: 1.3.0
Build Commit: 89f48353959c9b08608b6d7160c1f1c5ae2763ee
Build Timestamp: 2025-03-27T14:19:05Z
Build Hostname: 
Go Version: go1.24.0
Platform: linux/amd64
Rego Version: v1
WebAssembly: available


#### Install Required Python Packages

In [21]:
# 📦 Step 1: Install Python Packages Required for the Scenario Setup
!pip install ansible passlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 33.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 7.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.3 MB/s eta 0:00:00:00:01


In [22]:
%cd /home/wsuser/work/ITBench-CISO-CAA-Agent/

# Install the Python Packages required by the agent
!pip install -r requirements-dev.txt

/home/wsuser/work/ITBench-CISO-CAA-Agent
Ignoring async-timeout: markers 'python_version == "3.10"' don't match your environment
Ignoring exceptiongroup: markers 'python_version == "3.10"' don't match your environment
Ignoring pyreadline3: markers 'python_version >= "3.10" and python_version < "3.13" and sys_platform == "win32"' don't match your environment
Ignoring pywin32: markers 'python_version >= "3.10" and python_version < "3.13" and (sys_platform == "win32" or platform_system == "Windows")' don't match your environment
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 32.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 9.4 M

## Run Scenario and Agent

### Run the Scenario Setup

In [23]:
scenario_dir = "/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa"

# 📍 Change directory to the scenario
%cd {scenario_dir}

/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa


#### Deploy the Bundle

In [24]:
# ▶️ Run the Makefile target as in the container
!FOREGROUND=true make deploy_bundle

Using provided input file
make[1]: Entering directory '/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa'
Using provided input file
ansible-playbook -i dynamic_inventory.py ./playbooks/deploy.yml   

PLAY [Deploy the environment] **************************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: sftp transfer mechanism failed on [169.44.147.111]. Use
ANSIBLE_DEBUG=1 to see detailed information
[WARNING]: scp transfer mechanism failed on [169.44.147.111]. Use
ANSIBLE_DEBUG=1 to see detailed information
[WARNING]: Platform linux on host RHEL9 Machine is using the discovered Python
interpreter at /usr/bin/python3.9, but future installation of another Python
interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.18/reference_appendices/interpreter_discovery.html for more information.
ok: [RHEL9 Machine]

TASK [include_tasks] **************************************

#### Inject the Fault

In [25]:
# ▶️ Run the Makefile target as in the container
!FOREGROUND=true make inject_fault

Using provided input file
make[1]: Entering directory '/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa'
Using provided input file
ansible-playbook -i dynamic_inventory.py ./playbooks/execute.yml   

PLAY [Execute fault injection] *************************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: sftp transfer mechanism failed on [169.44.147.111]. Use
ANSIBLE_DEBUG=1 to see detailed information
[WARNING]: scp transfer mechanism failed on [169.44.147.111]. Use
ANSIBLE_DEBUG=1 to see detailed information
[WARNING]: Platform linux on host RHEL9 Machine is using the discovered Python
interpreter at /usr/bin/python3.9, but future installation of another Python
interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.18/reference_appendices/interpreter_discovery.html for more information.
ok: [RHEL9 Machine]

TASK [include_tasks] *************************************

### Run the Agent

Set correct permissions on copied ssh key

In [26]:
!chmod 600 /tmp/agent/id_rsa.ansible_user

Run the agent

In [27]:
%cd /home/wsuser/work/ITBench-CISO-CAA-Agent/src/ciso_agent/

import main

main.main(output="/home/wsuser/work/agent_workdir/result.json", goal = 
"""I would like to check if the following condition is satisfiled, given a host name `rhel9_servers`, on RHEL by using Ansible Playbook and OPA Rego policy
    Ensure SSH X11 forwarding is disabled 

To check the condition, do the following steps. 
- collect configuration from the RHEL host and save it locally. you can use ansible-playbook to do that. 
- chcek if the condition is met by using rego policy with the input given by the step above. 

for those steps, you need to create ansible playbook `playbook.yml` and OPA rego policy `policy.rego`.
If you can fix the generated code, do it and run the fixed code again.
Once you get a final answer, you can quit the work.

The Ansible inventory file (inventory.ansible.ini) for the target RHEL server is located at `/tmp/agent/inventory.ansible.ini`.

You can use `/tmp/agent\`""")

/home/wsuser/work/ITBench-CISO-CAA-Agent/src/ciso_agent
 # Goal:
I would like to check if the following condition is satisfiled, given a host name `rhel9_servers`, on RHEL by using Ansible Playbook and OPA Rego policy
    Ensure SSH X11 forwarding is disabled 

To check the condition, do the following steps. 
- collect configuration from the RHEL host and save it locally. you can use ansible-playbook to do that. 
- chcek if the condition is met by using rego policy with the input given by the step above. 

for those steps, you need to create ansible playbook `playbook.yml` and OPA rego policy `policy.rego`.
If you can fix the generated code, do it and run the fixed code again.
Once you get a final answer, you can quit the work.

The Ansible inventory file (inventory.ansible.ini) for the target RHEL server is located at `/tmp/agent/inventory.ansible.ini`.

You can use `/tmp/agent\`

Task Selection Result: rhel_playbook_opa
# Agent: Test
## Task: Check a rego policy for a given input fil

### Evaluate and Cleanup

In [28]:
# 📍 Change directory to the scenario
%cd {scenario_dir}

/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa


/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Evaluate the Agent

In [29]:
# ▶️ Run the Makefile target as in the container
!FOREGROUND=true make evaluate

Using provided input file
/tmp/agent/input.json
make[1]: Entering directory '/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa'
Using provided input file
ansible-playbook -i dynamic_inventory.py ./playbooks/evaluate.yml --extra-vars path_to_output=/tmp/agent/evaluation.json   

PLAY [Evaluate the system compliance] ******************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [include_tasks] ***********************************************************
included: /home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa/tasks/precondition_setup.yml for localhost

TASK [Check if the directory exists and create it if necessary] ****************
ok: [localhost]

TASK [Check if /tmp/agent/agent_output.data exists] ****************************
ok: [localhost]

TASK [Ensure dest directory exists] ********************************************
skipping: [localhost]

TASK [Extract /tmp/agent/agent_out

#### Cleanup the Scenario

In [30]:
# ▶️ Run the Makefile target as in the container
!FOREGROUND=true make destroy_bundle

Using provided input file
make[1]: Entering directory '/home/wsuser/work/ITBench/ciso/3.gen-cis-b-rhel9-ansible-opa'
Using provided input file
ansible-playbook -i dynamic_inventory.py ./playbooks/destroy.yml   

PLAY [Destroy the environment] *************************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: sftp transfer mechanism failed on [169.44.147.111]. Use
ANSIBLE_DEBUG=1 to see detailed information
[WARNING]: scp transfer mechanism failed on [169.44.147.111]. Use
ANSIBLE_DEBUG=1 to see detailed information
[WARNING]: Platform linux on host RHEL9 Machine is using the discovered Python
interpreter at /usr/bin/python3.9, but future installation of another Python
interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.18/reference_appendices/interpreter_discovery.html for more information.
ok: [RHEL9 Machine]

TASK [include_tasks] *************************************

Now you can rerun the setup and agent if you wish